# 여드름 종류 진단 AI

### 라이브러리 Import

In [1]:
import numpy as np
import pandas as pd
import os
import cv2
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model

### 공통 환경 변수

In [8]:
# 순서대로 정상, 좁쌀, 화농성, 염증성, 모낭염
labels = ['normal', 'comedones', 'pustules', 'papules', 'folliculitis']
data_dir = '/Users/jeongjin/Desktop/Git/AcneLog_AI/data'
image_size = (224, 224)

labels

['normal', 'comedones', 'pustules', 'papules', 'folliculitis']

### 데이터 로드 및 전처리

In [11]:
def load_images_and_labels(data_dir, labels, image_size=(224, 224)):
    image_dir = os.path.join(data_dir, 'image')
    images, targets = [], []
    for idx, label in enumerate(labels):
        folder_path = os.path.join(image_dir, label)
        for file in os.listdir(folder_path):
            if not file.lower().endswith(('.jpg', '.jpeg', '.png')):
                continue
            img_path = os.path.join(folder_path, file)
            img = cv2.imread(img_path)
            if img is None:
                continue
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, image_size)
            img = img / 255.0
            images.append(img)
            targets.append(idx)
    return np.array(images, dtype=np.float32), np.array(targets)

X, y = load_images_and_labels(data_dir, labels, image_size)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

num_classes = len(labels)
y_train = tf.keras.utils.to_categorical(y_train, num_classes=num_classes)
y_val = tf.keras.utils.to_categorical(y_val, num_classes=num_classes)

ValueError: With n_samples=1, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

### 데이터 증강 (Data Augmentation)

In [12]:
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    zoom_range=0.2
    # rescale=1./255  ← X_train을 미리 정규화했으므로 생략
)

# datagen.fit(X_train)  ← whitening 안 쓰면 생략 OK

### 모델 구성

In [13]:
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = Flatten()(base_model.output)
x = Dense(128, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(4, activation='softmax')(x)  # 4개 클래스
model = Model(inputs=base_model.input, outputs=x)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


### 모델 학습

In [14]:
history = model.fit(datagen.flow(X_train, y_train, batch_size=32),
                    validation_data=(X_val, y_val),
                    epochs=20,
                    callbacks=[tf.keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)])

NameError: name 'X_train' is not defined

### 모델 평가, 모델 저장 및 배포 설정

In [ ]:
val_loss, val_acc = model.evaluate(X_val, y_val)
print(f"Validation Accuracy: {val_acc:.2f}")

model.save("./acne_classifier.h5")